In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4

In [3]:
f = netCDF4.Dataset('2mTemperature.nc', 'r')
print(f)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2021-01-02 06:08:56 GMT by grib_to_netcdf-2.16.0: /opt/ecmwf/eccodes/bin/grib_to_netcdf -S param -o /cache/data4/adaptor.mars.internal-1609567732.693108-4091-30-76b0d09d-19d4-4ce5-8dc2-987bc0140edc.nc /cache/tmp/76b0d09d-19d4-4ce5-8dc2-987bc0140edc-adaptor.mars.internal-1609567732.6937065-4091-11-tmp.grib
    dimensions(sizes): longitude(3600), latitude(1801), time(7)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 t2m(time,latitude,longitude)
    groups: 



In [4]:
print(f.variables)

OrderedDict([('longitude', <class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (3600,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('latitude', <class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (1801,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (7,)
filling on, default _FillValue of -2147483647 used
), ('t2m', <class 'netCDF4._netCDF4.Variable'>
int16 t2m(time, latitude, longitude)
    scale_factor: 0.0014182848031955274
    add_offset: 269.1460956671687
    _FillValue: -32767
    missing_value: -32767
    units: K
    long_name: 2 metre temperature
unlimited d

In [5]:
ds = xr.open_dataset("2mTemperature.nc")

In [6]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 1801, longitude: 3600, time: 7)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2019-12-01T12:00:00 ... 2019-12-07T12:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-01-02 06:08:56 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...